In [24]:
import pandas as pd

## DATA SETS ---------
smsData = pd.read_csv("spam.csv", encoding = "latin-1", usecols=["v1", "v2"])## importa o data set como um data-frame, usa o latin-1 com decoding para ler o ficheiro e seleciona as colunas v1 e v2 como as unicas a ser usadas (o ficheiro tem mais 3 colunas mas vazias) 

## TRANSFORMS
smsData.columns = ["Status", "Mensagem"] #renomeia os hearders das colunas

print("Linhas x Colunas", smsData.shape) #print do tamanho da matriz
smsData.head()  # print das primeiras 5 linhas da mat

Linhas x Colunas (5572, 2)


,Status,Mensagem
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [43]:
print("Distribuição")
print(smsData['Status'].value_counts()) #Quantidade da ham x spam na tabela
print("")
print("Frequência Relativa")
print(smsData['Status'].value_counts(normalize=True)) #Frequencia relativa

Distribuição
ham     4825
spam     747
Name: Status, dtype: int64

Frequência Relativa
ham     0.865937
spam    0.134063
Name: Status, dtype: float64


In [60]:
#Divisão em Sets
randomSample = smsData.sample(frac=1, random_state=666) # Faz suffle da tabela - frac é a fração dos dados a usar (0,0 a 1,0) - random-state = (int) é a seed que o sistema vai usar como "fator de entropia" para gerar o "random"

# Definir o percentagem de dados a usar (0 a 1)
sampleSize = round(len(randomSample) * 0.8)

# Criação do grupo de teste e treino
trainGroup = randomSample[:sampleSize].reset_index(drop=True) #Faz splice dos dados com base no <inserir nome> que serve como indicador da percentagem de dados a usar. neste caso os primeiros 80% da tabela - reset_index faz reset ao indice da tabela
test_set = randomSample[sampleSize:].reset_index(drop=True) #Faz splice dos dados com base no <inserir nome> que serve como indicador da percentagem de dados a usar. neste caso os dados que sobram após "ler" 80% da tabela - reset_index faz reset ao indice da tabela


print("Dados de Treino")
print(trainGroup['Status'].value_counts())
print("")
print(trainGroup['Status'].value_counts(normalize = True))

print("")

print("Dados de Teste")
print(test_set['Status'].value_counts())
print("")
print(test_set['Status'].value_counts(normalize = True))

Dados de Treino
ham     3853
spam     605
Name: Status, dtype: int64

ham     0.864289
spam    0.135711
Name: Status, dtype: float64

Dados de Teste
ham     972
spam    142
Name: Status, dtype: int64

ham     0.872531
spam    0.127469
Name: Status, dtype: float64


In [61]:
#Limpeza dos dados
trainGroup['Mensagem'] = trainGroup['Mensagem'].str.replace('\W', ' ') # Limpa a pontuação
trainGroup['Mensagem'] = trainGroup['Mensagem'].str.lower()  #Transforma tudo em letra pequena
trainGroup.head()

,Status,Mensagem
0,ham,lol no ouch but wish i d stayed out a bit longer
1,ham,well done and luv ya all
2,ham,me 2 babe i feel the same lets just 4get abou...
3,ham,gal n boy walking in d park gal can i hold ur...
4,ham,sorry man my stash ran dry last night and i c...


In [62]:
trainGroup['Mensagem'] = trainGroup['Mensagem'].str.split() 

#Transformamos cada mensagem na coluna "Mensagem" numa lista de palavras - Usamos o split para dividir a frase em várias palavras com o espaço como identificador do fim de cada palavra.

palavras = [] 
for mensagem in trainGroup['Mensagem']:
   for word in mensagem:
      palavras.append(word)

palavras = list(set(palavras))

# Com os nested fors vamos adicionando à lista "palavras" cada palavra presente nas várias mensagens. Depois usamos o set para remover todas as palavras duplicadas da lista e finalmente convertemos a lista de volta a uma lista com o cast do list porque no for são convertidas em string.

print("Total de palavras: ", len(palavras))

Total de palavras:  7819


In [80]:
wordCounter = {}
wordCounter = {unique_word: [0] * len(trainGroup['Mensagem']) for unique_word in palavras}

for index, mensagem in enumerate(trainGroup['Mensagem']):
   for word in mensagem:
      wordCounter[word][index] += 1

wordTable = pd.DataFrame(wordCounter)
wordTable.head()

# Criamos um dicionário em que cada chave é uma palavra da lista "palavra". Depois o for vai popular a tabela com as várias palavras (colunas) e a frequencia delas em cada um dos elementos do CSV (linhas). Na tabela abaixo só aparecem as primeiras 5 linhas porque o default do .head() é 5


,gardener,dare,bowls,pussy,misfits,messages,121,09066382422,eng,coincidence,...,lovable,supreme,nig,grief,0125698789,bday,sec,atural,fantasy,phyhcmk
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
trainGroup_processed = pd.concat([trainGroup, wordTable], axis=1)
trainGroup_processed.head()

# Juntamos a "tabela" do trainGroup com a da contagem de palavras para ficarmos com um data frame bonito e limpo que podemos usar para trainar o nosso modelo.

,Status,Mensagem,gardener,dare,bowls,pussy,misfits,messages,121,09066382422,...,lovable,supreme,nig,grief,0125698789,bday,sec,atural,fantasy,phyhcmk
0,ham,"[lol, no, ouch, but, wish, i, d, stayed, out, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[well, done, and, luv, ya, all]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[me, 2, babe, i, feel, the, same, lets, just, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[gal, n, boy, walking, in, d, park, gal, can, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[sorry, man, my, stash, ran, dry, last, night,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
#Splice 2. Electric Splicealoo - Separamosas mensagens em 2 grupos - ham e spam - para podermos realizar o calculo
ham_messages = trainGroup_processed[trainGroup_processed['Status'] == 'ham']
spam_messages = trainGroup_processed[trainGroup_processed['Status'] == 'spam'] 


# Probabilidades
p_spam = len(spam_messages) / len(trainGroup_processed)
p_ham = len(ham_messages) / len(trainGroup_processed)

#n do Spam é igual à soma todas as palavras de todas as mensagens Spam
totalWordsPerSpam = spam_messages['Mensagem'].apply(len)
n_spam = totalWordsPerSpam.sum()

#n do Ham é igual à soma todas as palavras de todas as mensagens Ham
totalWordsPerHam = ham_messages['Mensagem'].apply(len)
n_ham = totalWordsPerHam.sum()

# Auto explicativo
n_vocabulary = len(palavras)

# Laplace smoothing
alpha = 1

In [68]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in palavras}
parameters_ham = {unique_word:0 for unique_word in palavras}

# Calculate parameters
for word in palavras:
   n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
   p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
   parameters_spam[word] = p_word_given_spam

   n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
   p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
   parameters_ham[word] = p_word_given_ham

In [69]:
import re

def classify(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham: 
         p_ham_given_message *= parameters_ham[word]

   print('P(Spam|message):', p_spam_given_message)
   print('P(Ham|message):', p_ham_given_message)

   if p_ham_given_message > p_spam_given_message:
      print('Label: Ham')
   elif p_ham_given_message < p_spam_given_message:
      print('Label: Spam')
   else:
      print('Equal proabilities, have a human classify this!')

In [70]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.0708607526070652e-25
P(Ham|message): 3.248694784512615e-27
Label: Spam


In [71]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 4.153907788533206e-25
P(Ham|message): 1.304467526392837e-21
Label: Ham


In [72]:
def classify_test_set(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham:
         p_ham_given_message *= parameters_ham[word]

   if p_ham_given_message > p_spam_given_message:
      return 'ham'
   elif p_spam_given_message > p_ham_given_message:
      return 'spam'
   else:
      return 'needs human classification'

In [73]:
test_set['predicted'] = test_set['Mensagem'].apply(classify_test_set)
test_set.head()

,Status,Mensagem,predicted
0,ham,"Nothing, i got msg frm tht unknown no..",ham
1,ham,Yup it's at paragon... I havent decided whethe...,ham
2,ham,Do u still have plumbers tape and a wrench we ...,ham
3,ham,"Babe, I'm answering you, can't you see me ? Ma...",ham
4,spam,"ou are guaranteed the latest Nokia Phone, a 40...",spam


In [74]:
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
   row = row[1]
   if row['Status'] == row['predicted']:
      correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1103
Incorrect: 11
Accuracy: 0.9901256732495511


In [76]:
classify("OMG you're the winner. Lucky prize Tao Bao")

P(Spam|message): 5.044308548608223e-27
P(Ham|message): 3.5798508608331393e-29
Label: Spam
